In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import datetime 
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import sqlalchemy

init_notebook_mode(connected=True)

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

WINDOW = 22

Using TensorFlow backend.


In [3]:
"""
Парсинг данных

"""
def connect(user, password, db, host: str, port: int, echo=True):
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    eng = sqlalchemy.create_engine(url, client_encoding='utf8', echo=echo)
    meta = sqlalchemy.MetaData(bind=eng)

    return eng, meta


def get_data_frame(pair: str = 'USDT_BTC') -> pd.DataFrame:
    """Метод стягивания данных из базы в датафрейм.
    По умолчанию тянет все значения в базе для валютной пары доллар биткоин.
    Список спаршенных пар смотри в таблице Pair
    Цепляться будет отовсюду где есть инетрнет"""
    engine, meta = connect(user='postgres', password='password', db='btccandles', host='176.212.125.30', port=16432)
    df = pd.read_sql_query(
        'select date, time, open, close, low, high, volume, pair."name" from candlestick, pair '
        'where candlestick.pair_id=pair.id and pair."name"=\'' + pair + '\';',
        con=engine)
    return df

"""
Загрузка данных
"""
def load_data(X, seq_len, train_size=1):
    
    amount_of_features = X.shape[1] 
    X_mat = X.as_matrix() 
    
    sequence_length = seq_len + 1 
    data = []
    
    for index in range(len(X_mat) - sequence_length):
        data.append(X_mat[index: index + sequence_length])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))

    return x_train, y_train

"""
Собираем LSTM
"""

def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

def predictNumDay(num, pathModel, data, scaler):
    m = load_model(pathModel)
    
    prediction = []
    lastbatch = np.array(data[-WINDOW:])
    for i in np.arange(num):    
        res = m.predict([lastbatch.reshape(1,22, 5)])
        prediction.append(scaler.inverse_transform(res))
    
    return np.array(prediction).reshape(num)


In [6]:
"""
typeBlockchain:
USDT_BTC
USDT_LTC
USDT_ETH
USDT_ETC
USDT_XRP
"""

def nextDayPrediction(typeBlockchain, N = 1):    
    
    df = get_data_frame(typeBlockchain)

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    all_df = df.copy()

    x = all_df[['open', 'low', 'high', 'volume']].copy()
    y = all_df['close'].copy()

    x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

    y = y_scaler.fit_transform(y.values.reshape(-1, 1))
    x['close'] = y
    
    X_train, y_train = load_data(x, WINDOW)
    
    #print (X_train.shape, y_train.shape)
    
    model = build_model(input_shape=(WINDOW, 5))
    
    print('START FIT MODEL...')
    
    start = time.time()
    #model.fit(X_train, y_train, batch_size=32, epochs=500,
    #          verbose=0)
    end = time.time()

    print ('Learning time: ', end-start)
    
    today = time.strftime("_%d_%m_%Y")
    
    pathModel = "../models/model_5f_" + typeBlockchain + today +".h5"
    #save_model(model, pathModel)
    
    model = load_model(pathModel)
    
    trainPredict = model.predict(X_train)
    trainPredict = y_scaler.inverse_transform(trainPredict)
    trainY = y_scaler.inverse_transform([y_train])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))
    
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    """
    
    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.legend(loc='best')
    plt.show()
    
    """
    
    lastDate =str(df.date[df.last_valid_index()]).split('-')
    currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])+1)
    predictionDate = pd.date_range(currentData,periods=N)
    predictNday =  (predictNumDay(N, pathModel, x, y_scaler))

    prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)
    
    Actual = pd.DataFrame(prices, columns=["close"], index=df.date).close
    Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
    pred = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close

    """ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
    TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

    iplot([ActualValues,TrainingValues])
    
    our_Predict = go.Scatter( x = prediction.index, y = prediction.predictionPrice, name = 'Next5DayValues')

    iplot([our_Predict, ActualValues])"""
    
    return prediction

In [7]:
USDT_BTC = nextDayPrediction('USDT_BTC', N = 1)

2017-07-19 20:27:27,879 INFO sqlalchemy.engine.base.Engine select version()
2017-07-19 20:27:27,880 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:27,881 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-19 20:27:27,882 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:27,883 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:27,884 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:27,885 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:27,886 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:27,887 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-19 20:27:27,888 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:27,890 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [8]:
USDT_LTC = nextDayPrediction('USDT_LTC', N = 1)

2017-07-19 20:27:33,959 INFO sqlalchemy.engine.base.Engine select version()
2017-07-19 20:27:33,960 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:33,962 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-19 20:27:33,962 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:33,964 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:33,964 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:33,967 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:33,967 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:33,968 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-19 20:27:33,969 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:33,970 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [9]:
USDT_ETH = nextDayPrediction('USDT_ETH', N = 1)

2017-07-19 20:27:41,836 INFO sqlalchemy.engine.base.Engine select version()
2017-07-19 20:27:41,836 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:41,838 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-19 20:27:41,838 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:41,840 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:41,840 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:41,841 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:41,842 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:41,843 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-19 20:27:41,843 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:41,845 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [10]:
USDT_ETC = nextDayPrediction('USDT_ETC', N = 1)

2017-07-19 20:27:51,356 INFO sqlalchemy.engine.base.Engine select version()
2017-07-19 20:27:51,357 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:51,358 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-19 20:27:51,359 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:51,360 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:51,361 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:51,362 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:27:51,363 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:51,364 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-19 20:27:51,364 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:27:51,366 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [5]:
USDT_XRP = nextDayPrediction('USDT_XRP', N = 1)

2017-07-19 20:19:13,267 INFO sqlalchemy.engine.base.Engine select version()
2017-07-19 20:19:13,269 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:19:13,272 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-19 20:19:13,273 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:19:13,275 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:19:13,276 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:19:13,279 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-19 20:19:13,279 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:19:13,281 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-19 20:19:13,282 INFO sqlalchemy.engine.base.Engine {}
2017-07-19 20:19:13,285 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [11]:
print ('USDT_BTC:\n', USDT_BTC)
print ('USDT_LTC:\n', USDT_LTC)
print ('USDT_ETH:\n', USDT_ETH)
print ('USDT_ETC:\n', USDT_ETC)
print ('USDT_XRP:\n', USDT_XRP)

USDT_BTC:
             predictionPrice
2017-07-20      2390.282471
USDT_LTC:
             predictionPrice
2017-07-20        42.804066
USDT_ETH:
             predictionPrice
2017-07-20        201.74791
USDT_ETC:
             predictionPrice
2017-07-20        15.185702
USDT_XRP:
             predictionPrice
2017-07-20         0.175407
